In [1]:
import os
import cv2
import torch
import math
import numpy as np
import pandas as pd
from ultralytics import YOLO
from collections import defaultdict
from deep_sort_realtime.deepsort_tracker import DeepSort
from utils.dataset_utils import get_swarm_data, get_quality_score, get_full_tracks, get_state_actions, get_specific_states_actions, shape_action_tensors, shape_state_tensors, save_state_action_tensors

In [2]:
# Analyse des Attention Network, wie viele lokalen Nachbarn werden wie einbezogen?
# - Referenzen das 1 - 2 nach Nachbarn reichen, oder 2-fache Fischlänge
# - So kann auch die Death-Rate ermittelt werden, aus Predator sicht die gewichte der Fische interpretieren

# References: 
# https://learnopencv.com/real-time-deep-sort-with-torchvision-detectors/#Real-Time-Deep-SORT-Setup
# https://pypi.org/project/deep-sort-realtime/

# Vergleich mit Daten aus Kalifornien und Analyse des Lebenstriebs

In [3]:
# Inputs
video = "video_5s"
clip_size = 10
num_fish = 33

In [4]:
raw_video_folder = r'..\data\raw\videos'
yolo_path = f'..\models\costumized_yolo\costumized_yolo\costumized_yolo.pt'
processed_video_folder = f'..\data\processed\{video}'

<string>:3: SyntaxWarning: invalid escape sequence '\{'
<>:2: SyntaxWarning: invalid escape sequence '\m'
<>:3: SyntaxWarning: invalid escape sequence '\{'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<string>:3: SyntaxWarning: invalid escape sequence '\{'
<>:2: SyntaxWarning: invalid escape sequence '\m'
<>:3: SyntaxWarning: invalid escape sequence '\{'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\janni\AppData\Local\Temp\ipykernel_17460\1604442491.py:2: SyntaxWarning: invalid escape sequence '\m'
  yolo_path = f'..\models\costumized_yolo\costumized_yolo\costumized_yolo.pt'
C:\Users\janni\AppData\Local\Temp\ipykernel_17460\1604442491.py:3: SyntaxWarning: invalid escape sequence '\{'
  processed_video_folder = f'..\data\processed\{video}'
C:\Users\janni\AppData\Local\Temp\ipykernel_17460\1604442491.py:3: SyntaxWarning: invalid escape sequence '\d'
  processed_video_folder = f'..\data\processed\{video}'


In [5]:
video_path = raw_video_folder + "\\" + video + ".mp4"
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = total_frames / fps
total_traj = total_frames - clip_size + 1

print(f"FPS: {fps}")
print(f"Total frames: {total_frames}")
print(f"Duration (s): {duration:.2f}")
print(f"Total trajectories: {total_traj}")
print("Prey Count: 32")
print("Predator Count: 1")

FPS: 30.0
Total frames: 153
Duration (s): 5.10
Total trajectories: 144
Prey Count: 32
Predator Count: 1


In [ ]:
device  = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model   = YOLO(yolo_path)
tracker = DeepSort(max_age=5)

video_idx   = 0
threshold = num_fish * 0.75

raw_path = os.path.join(processed_video_folder, "state_actions", "raw")
os.makedirs(raw_path, exist_ok=True)

for start_frame in range(total_traj):
    end_frame = start_frame + clip_size - 1
    print(f"[Clip {video_idx}] Startframe: {start_frame}, Endframe: {end_frame}")

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    frames = []
    for _ in range(clip_size):
        success, frame = cap.read()
        if not success:
            break
        frames.append(frame)
    if len(frames) < clip_size:
        break

    dfs = []
    for offset, frame in enumerate(frames):
        frame_idx = start_frame + offset
        df = get_swarm_data(frame, model, tracker, frame_idx)
        dfs.append(df)

    combined_df = pd.concat(dfs, ignore_index=True)

    mean_track_visibility, num_full_tracks, mean_confidence = get_quality_score(combined_df, n=clip_size)
    video_idx += 1

    full_tracks_df = get_full_tracks(combined_df, clip_size)

    num_tracks = int(len(full_tracks_df) / clip_size)
    
    if num_tracks > threshold:
        df_state_actions = get_state_actions(full_tracks_df)
        df_state_actions.to_csv(os.path.join(raw_path, f"{video_idx}_{video}_{num_tracks}_state_actions.csv"), index=False)

cap.release()

[Clip 0] Startframe: 0, Endframe: 9

0: 736x736 1 Predator, 1 Predator Head, 33 Preys, 63.9ms
Speed: 7.8ms preprocess, 63.9ms inference, 3.6ms postprocess per image at shape (1, 3, 736, 736)

0: 736x736 1 Predator, 1 Predator Head, 33 Preys, 14.8ms
Speed: 5.0ms preprocess, 14.8ms inference, 2.1ms postprocess per image at shape (1, 3, 736, 736)

0: 736x736 1 Predator, 1 Predator Head, 34 Preys, 12.4ms
Speed: 4.9ms preprocess, 12.4ms inference, 1.6ms postprocess per image at shape (1, 3, 736, 736)

0: 736x736 1 Predator, 1 Predator Head, 35 Preys, 12.2ms
Speed: 4.6ms preprocess, 12.2ms inference, 1.6ms postprocess per image at shape (1, 3, 736, 736)

0: 736x736 1 Predator, 1 Predator Head, 35 Preys, 13.7ms
Speed: 5.6ms preprocess, 13.7ms inference, 1.6ms postprocess per image at shape (1, 3, 736, 736)

0: 736x736 1 Predator, 1 Predator Head, 34 Preys, 14.5ms
Speed: 4.7ms preprocess, 14.5ms inference, 2.0ms postprocess per image at shape (1, 3, 736, 736)

0: 736x736 1 Predator, 1 Predator

In [7]:
raw_path = os.path.join(processed_video_folder, "state_actions", "raw")
os.makedirs(raw_path, exist_ok=True)

for name in os.listdir(raw_path):
    file_path = os.path.join(raw_path, name)
    df_full = pd.read_csv(file_path)

    n_agents = df['agent_id'].nunique()

    df_pred_prey = get_specific_states_actions(df_full, "Predator Head", "Prey")
    df_prey_prey = get_specific_states_actions(df_full, "Prey", "Prey")
    df_prey_pred = get_specific_states_actions(df_full, "Prey" ,"Predator Head")

    state_actions_path = os.path.join(processed_video_folder, "state_actions")
    os.makedirs(state_actions_path, exist_ok=True)

    dfs = [("df_full", df_full),
           ("df_pred_prey", df_pred_prey),
           ("df_prey_prey", df_prey_prey),
           ("df_prey_pred", df_prey_pred),]

    for name, df in dfs:
        folder_path = os.path.join(state_actions_path, name)
        os.makedirs(folder_path, exist_ok=True)

        state_tensors = []
        action_tensors = []

        for i in df.iloc():
            state_array = df[['dx', 'dy', 'rel_v_x', 'rel_v_y']].to_numpy(dtype=np.float32)
            action_array = df['theta'].to_numpy(dtype=np.float32)

            state_tensor = torch.from_numpy(state_array)
            action_tensor = torch.from_numpy(action_array).unsqueeze(1)

            state_tensors.append(state_tensor)
            action_tensors.append(action_tensor)

        # Change format
        states = shape_state_tensors(state_tensors) # (batch, n_pairs, seq_len, state_dim)
        actions = shape_action_tensors(action_tensors) # (batch, n_pairs, seq_len, action_dim)

        torch.save(states,  os.path.join(folder_path, f"(Agents {n_agents})_{name}_states.pt"))
        torch.save(actions, os.path.join(folder_path, f"(Agents {n_agents})_{name}_actions.pt"))